In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import math
import hashlib

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# These will already be installed in the testing environment so disregard the
# amount of time (~1 minute) it takes to install.
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 33 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack ..

In [3]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [4]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 30000000

def handler(signum, frame):
  raise Exception("Authentication timeout!")

signal.signal(signal.SIGALRM, handler)
signal.alarm(AUTH_TIMEOUT)

try:
   auth.authenticate_user()
except:
   pass

In [5]:
# Copy one wikidumps files
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP
project_id = 'ass3-414106'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata20210801_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/"

Updated property [core/project].
Copying gs://wikidata20210801_preprocessed/multistream1_preprocessed.parquet...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

- [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [6]:
from pathlib import Path
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd


In [7]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id, withTF=False):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  size = 0
  word_counts = Counter()

  for token in tokens:
      if token not in all_stopwords:
          word_counts[token] += 1
          size += 1

  # Create tuples of (token, (doc_id, tf))
  if(not withTF):
    result = [(token, (id, count)) for token, count in word_counts.items()]
  else:
    result = [(token, (id, count/size)) for token, count in word_counts.items()]

  return result


In [8]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_TF = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1],True))

word_counts.take(1)

[('anarchism', (12, 125))]

In [9]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''


  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])

  return sorted_pl



In [10]:
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_TF = word_TF.groupByKey().mapValues(reduce_word_counts)

# postings.take(10)

In [11]:
postings_filtered = postings.filter(lambda x: len(x[1])>10)
postings_TF_filtered = postings_TF.filter(lambda x: len(x[1])>10)

In [12]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''

  return postings.map(lambda x: (x[0], len(x[1])))

In [13]:
w2df = calculate_df(postings_filtered)
# w2df.take(1)
w2df_dict = w2df.collectAsMap()

In [14]:
N = 1000;
postings_TF_IDF_filtered = postings_TF_filtered.map(lambda x: (x[0], updated_value_function(x[0],x[1])))

def updated_value_function(term, post):
    return list(map(lambda x: (x[0], x[1]*math.log10(N/w2df_dict[term])), post))

In [17]:
postings_TF_IDF_filtered_dict = postings_TF_IDF_filtered.collectAsMap()
print(postings_TF_IDF_filtered_dict["political"])

[(12, 0.003344736074330496), (303, 0.0003245996872968506), (307, 0.0015856517205880612), (308, 0.0008703286418990512), (336, 0.00013827211988195576), (339, 0.0025156855503090498), (358, 0.001195733990438265), (359, 0.0004114825886728235), (569, 0.0025156855503090498), (594, 5.292921158817012e-05), (600, 0.0008226503475149868), (620, 0.0015468537857994985), (624, 0.00037019832123746314), (627, 0.00033253732088724363), (643, 0.0017511465866483515), (663, 0.0002370379197976384), (665, 0.0006968269631899467), (676, 0.00015919514424075923), (677, 0.00024888981578752035), (689, 0.0004691116946780488), (690, 0.0003631219196899993), (691, 0.0012649458872965738), (700, 0.00043215595024199673), (701, 0.0019672727954462687), (705, 0.0022974444534232647), (706, 0.0002921750011418717), (717, 0.0003367378653937012), (736, 0.00022081133711707718), (737, 0.0005847737513789455), (738, 0.0005242624356820235), (740, 0.0003073767879365603), (746, 0.0006974597847704254), (751, 0.00021668762680412897), (752

In [18]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''

  # Assign each token to a bucket
  bucketed_postings = postings.map(lambda x: (token2bucket_id(x[0]), [(x[0], x[1])]))

  # Group the postings by bucket
  grouped_postings = bucketed_postings.reduceByKey(lambda x, y: x + y)

  # Write out each bucket and collect posting locations
  posting_locations = grouped_postings.map(lambda x: InvertedIndex.write_a_posting_list(x))

  return posting_locations

In [19]:
posting_locs_list = partition_postings_and_write(postings).collect()
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)


In [21]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.tfidf = postings_TF_IDF_filtered_dict;
inverted.write_index('.', 'index')


In [26]:
inverted_index = InvertedIndex.read_index('.', 'index')

print(inverted_index._posting_lists)

AttributeError: 'InvertedIndex' object has no attribute '_posting_lists'